### Importação dos metadados

In [ ]:
from src import transformer
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import nltk
%load_ext autoreload
%autoreload 2
df = pd.read_csv("dados/corpus_tratado/metadados.csv")[0:500]

### Filtrando os ids por assuntos

In [ ]:
corte = 50
df_validos = pd.DataFrame(df.groupby("Assunto").size()).reset_index()
df_validos.columns = ["assunto", "quant"]
df_validos = df_validos.loc[df_validos.quant >= corte]
display(df_validos.quant.sum())
display(df_validos.assunto.nunique())
documentos_validos = df[df.Assunto.isin(df_validos.assunto)][["id", "Assunto"]].reset_index().drop('index', axis = 1)
diretorio = "dados/corpus_tratado/"
documentos_validos['arquivo'] = documentos_validos.id + '.txt'
documentos_validos['teores'] = [transformer.recuperar_teor(x, diretorio) for x in tqdm(documentos_validos['arquivo'])]
documentos_validos

### cortando documentos curtos

In [ ]:
min_palavras_documento = 50
documentos_validos['validos'] = documentos_validos.teores.apply(lambda x: 0 if len(x.split(' ')) <= min_palavras_documento else 1)
documentos_validos = documentos_validos[documentos_validos['validos'] == 1].drop(['arquivo', 'teores', 'validos'], axis = 1).reset_index().drop('index', axis = 1)
documentos_validos

### Execução do fluxo de experimentação

In [ ]:
n_experimentos = 1
lista_k = np.arange(2,10)
grid_minfreqs = [0, 50, 100]
grid_stopwords = [True, False]
grid_ica = [True, False]
grid_tesauro = [True, False]
grid_dimensoes = [100,300]
df = transformer.transform_param(documentos_validos, n_experimentos, grid_minfreqs, grid_stopwords, grid_ica, grid_tesauro, grid_dimensoes, lista_k)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
lista_k = [2, 3, 4]
plt.rcParams.update(plt.rcParamsDefault)
dir_experimento = 'experimento___minfreq_0__com_crit_tesauro__com_crit_ica__removeu_sw_pt__100_dims__1'
le = LabelEncoder()
modelo = 'vec_w2v_ger_soma'
df = pd.read_csv('dados/'+dir_experimento+'/vetores_teste.csv')
df[modelo] = df[modelo].apply(lambda x: transformer.converter_string_array(x))
X_kmeans = np.stack(df[modelo])
X_kmeans = X_kmeans.reshape(X_kmeans.shape[0], X_kmeans.shape[2])
y_kmeans = df['assunto']
le.fit(y_kmeans)
y_kmeans = le.transform(y_kmeans)
lista_scores_k = transformer.computar_scores_agrupamento(X_kmeans, y_kmeans, dir_experimento, modelo, lista_k)
transformer.gerar_graficos_kmeans(lista_scores_k, dir_experimento, modelo)
np.save(dir_experimento + '/lista_scores_k.npy', lista_scores_k)

In [ ]:
X_kmeans = np.stack(df[modelo])
X_kmeans = X_kmeans.reshape(X_kmeans.shape[0], X_kmeans.shape[2])
X_kmeans

In [ ]:
df[modelo]